# 1. Initialize SparkSession and load data

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CreditCardFraud_StructuredAPI").getOrCreate()

df = spark.read.csv("hdfs:///hcmus/22120227/Exercises/Lab3/data/creditcard.csv", header=True, inferSchema=True)
df.printSchema()
print(f"Total rows = {df.count()}")

your 131072x1 screen size is bogus. expect trouble
25/05/10 13:17:47 WARN Utils: Your hostname, HuuNghia-PC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/10 13:17:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 13:17:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

# 2. Deduplicate & Missing-Value Check

## 2.1. Remove dulicates

In [2]:
n_before = df.count()

df = df.dropDuplicates()  
n_after  = df.count()
print(f"Deduplicated: removed {n_before - n_after} duplicate rows; now {n_after} rows remain.")

25/05/10 13:17:54 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Deduplicated: removed 1081 duplicate rows; now 283726 rows remain.


## 2.2. Check for missing values

In [3]:
from pyspark.sql import functions as F

missing_counts = df.select([
    F.count(F.when(F.col(c).isNull() | F.isnan(F.col(c)), c)).alias(c)
    for c in df.columns
])
missing_counts.show()

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|   0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+



# 3. Feature Assembly & Scaling

In [4]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Scale Time & Amount
assembler_ta = VectorAssembler(inputCols=["Time", "Amount"], outputCol="ta_vec")
scaler_ta = StandardScaler(inputCol="ta_vec", outputCol="ta_scaled", withMean=True, withStd=True)

# Assemble V1–V28 + ta_scaled into vector
assembler_feats = VectorAssembler(inputCols=[f"V{i}" for i in range(1,29)] + ["ta_scaled"], outputCol="features")

preproc = Pipeline(stages=[assembler_ta, scaler_ta, assembler_feats])
preproc_model = preproc.fit(df)
df_feats = preproc_model.transform(df).select("features","Class")
df_feats.show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                        

# 4. Split into train and test sets (80/20)

In [5]:
train_df, test_df = df_feats.randomSplit([0.8, 0.2], seed=42)
print(f"Training count: {train_df.count()}, Test count: {test_df.count()}")

Training count: 227364, Test count: 56362


# 5. Model Training

In [6]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="Class", maxIter=20)
model = lr.fit(train_df)

25/05/10 13:18:08 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


# 6. Inspect Model Parameters

In [7]:
coeff  = model.coefficients.toArray().tolist()
interc = model.intercept
print(f"Intercept: {interc:.4f}")
print("Top 5 Coefficients:", coeff[:5], "…")

Intercept: -8.6098
Top 5 Coefficients: [0.09607245925312342, -0.025869081541789372, -0.014673681899441574, 0.67517406267906, 0.0934677780903822] …


# 7. Evaluation

In [8]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

pred = model.transform(test_df)

# AUC (ROC) and AUC (PR)
e_roc = BinaryClassificationEvaluator(
    labelCol="Class", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
e_pr  = BinaryClassificationEvaluator(
    labelCol="Class", rawPredictionCol="rawPrediction", metricName="areaUnderPR")

roc_auc = e_roc.evaluate(pred)
pr_auc  = e_pr.evaluate(pred)

# Accuracy, Precision, Recall
e_acc = MulticlassClassificationEvaluator(
    labelCol="Class", predictionCol="prediction", metricName="accuracy")
e_prec = MulticlassClassificationEvaluator(
    labelCol="Class", predictionCol="prediction", metricName="precisionByLabel")
e_rec  = MulticlassClassificationEvaluator(
    labelCol="Class", predictionCol="prediction", metricName="recallByLabel")

acc      = e_acc.evaluate(pred)
precision= e_prec.evaluate(pred, {e_prec.metricLabel: 1.0})
recall   = e_rec.evaluate(pred,  {e_rec.metricLabel: 1.0})

print(f"ROC-AUC = {roc_auc:.4f}")
print(f"PR-AUC  = {pr_auc:.4f}")
print(f"Accuracy  = {acc:.4f}")
print(f"Precision = {precision:.4f}")
print(f"Recall    = {recall:.4f}")

ROC-AUC = 0.9850
PR-AUC  = 0.7498
Accuracy  = 0.9992
Precision = 0.8800
Recall    = 0.6346


# 8. Export Test Set with All Original Columns + Prediction

In [9]:
from pyspark.sql.functions import monotonically_increasing_id

# Split the original preprocessed DataFrame into train/test
train_df, test_df = df_feats.randomSplit([0.8, 0.2], seed=42)
print(f"Test set size: {test_df.count()}")

# Define and train logistic regression model
lr = LogisticRegression(featuresCol="features", labelCol="Class", maxIter=20)
model = lr.fit(train_df)

# Make predictions on the test set
predictions_df = model.transform(test_df)

# Reattach raw features for CSV export
raw_cols = ["Time"] + [f"V{i}" for i in range(1, 29)] + ["Amount", "Class"]
full_df = df.select(*raw_cols)

# Add row indices for join
test_df_indexed = test_df.withColumn("row_id", F.monotonically_increasing_id()).withColumnRenamed("Class", "label")
full_df_indexed = full_df.withColumn("row_id", monotonically_increasing_id())

# Join to recover original columns + prediction
joined = test_df_indexed.join(full_df_indexed, on="row_id").join(
    predictions_df.select("features", "prediction", "probability"), on="features"
)

# Select required columns
output_df = joined.select("Time", *[f"V{i}" for i in range(1, 29)], "Amount", "label", "prediction")

# Export to CSV
output_df.coalesce(1).write.csv("results", header=True, mode="overwrite")

Test set size: 56362


## Combine into a single file

In [10]:
output_path = "results"

# Coalesce the DataFrame to a single partition so it generates only one CSV part file
output_df.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

# Move the generated part file to a fixed single CSV path
import os
import shutil

for fname in os.listdir(output_path):
    if fname.startswith("part-") and fname.endswith(".csv"):
        src_path = os.path.join(output_path, fname)
        dest_path = "results.csv"
        shutil.move(src_path, dest_path)
        break

# Remove the temporary output folder
shutil.rmtree(output_path)